In [ ]:
import pandas as pd
import numpy as np
import tqdm
from scipy import optimize
import sys
sys.path.append('/n/groups/price/jordan/h2xancestry/scripts')
import ldsc_utils
import plotly.express as px
import os
from statsmodels.stats.meta_analysis import combine_effects

# Figure 3

In [ ]:
nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())

dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/' 
coarse_grid_enrichment = pd.concat([pd.read_csv(dir + t + '.coarse_grid_enrichment.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits])
#maf_bin_means = pd.read_csv('/n/groups/price/jordan/h2xancestry/data/ldscores/1000G_EUR_Phase3/maf_bins_stratified_by_afr_maf_corrected/maf_bin_means.csv')
coarse_grid_enrichment_meta = coarse_grid_enrichment.groupby(
    'Category'
).apply(lambda df: pd.DataFrame({
    'expectation_b2_normalized_meta' : [combine_effects(df.expectation_b2_normalized_point_estimate, df.expectation_b2_normalized_se ** 2, method_re = 'iterated').summary_frame().loc['random effect', 'eff']],
    'expectation_b2_normalized_meta_se' : [combine_effects(df.expectation_b2_normalized_point_estimate, df.expectation_b2_normalized_se ** 2, method_re = 'iterated').summary_frame().loc['random effect', 'sd_eff']]
})).reset_index(
).assign(
    eur_maf_bin = lambda df: df.Category.str.split('_', expand = True).iloc[:,4],
    afr_maf_bin = lambda df: df.Category.str.split('_', expand = True).iloc[:,7],
    expectation_b2_normalized_meta_ci_dist = lambda df: df.expectation_b2_normalized_meta_se * 1.96
)
coarse_grid_enrichment_meta_wide = coarse_grid_enrichment_meta.pivot(
    index = ['afr_maf_bin'],
    columns = 'eur_maf_bin',
    values = 'expectation_b2_normalized_meta'
)
px.imshow(coarse_grid_enrichment_meta_wide)
coarse_grid_enrichment_meta.to_csv(
    '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/coarse_grid_enrichment_meta.csv',
    index = False
)

In [ ]:
px.bar(
    coarse_grid_enrichment_meta[~pd.isna(coarse_grid_enrichment_meta.afr_maf_bin)],
    x = 'afr_maf_bin', 
    y = 'expectation_b2_normalized_meta',
    color = 'eur_maf_bin',
    barmode = 'group',
    error_y = 'expectation_b2_normalized_meta_ci_dist'
)

In [ ]:
nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())

dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/' 
maf_bin_means = pd.read_csv(
    '/n/groups/price/jordan/h2xancestry/data/ldscores/1000G_EUR_Phase3/maf_bins_stratified_by_afr_maf_corrected/maf_bin_means.csv'
).rename(
    columns = {'bin' : 'Category'}
)
maf_bin_enrichment = pd.concat(
    [pd.read_csv(dir + t + '.maf_bin_stratified_by_afr_maf_corrected_enrichment.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits]
).groupby(
    'Category'
).apply(lambda df: pd.DataFrame({
    'expectation_b2_normalized_meta' : [combine_effects(df.expectation_b2_normalized_point_estimate, df.expectation_b2_normalized_se ** 2, method_re = 'iterated').summary_frame().loc['random effect', 'eff']],
    'expectation_b2_normalized_meta_se' : [combine_effects(df.expectation_b2_normalized_point_estimate, df.expectation_b2_normalized_se ** 2, method_re = 'iterated').summary_frame().loc['random effect', 'sd_eff']]
})).reset_index(
).merge(
    maf_bin_means
).assign(
    snp_group = lambda df: df.Category.str.split('(', expand = True).iloc[:, 0]
).query(
    'snp_group.str.contains("quintile")'
)
maf_bin_enrichment.to_csv(
    '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/maf_bin_stratified_by_afr_maf_corrected_enrichment_meta.csv',
    index = False
)
#coarse_grid_enrichment_meta_wide = coarse_grid_enrichment_meta.pivot(
#    index = ['afr_maf_bin'],
#    columns = 'eur_maf_bin',
#    values = 'expectation_b2_normalized_meta'
#)
#px.imshow(coarse_grid_enrichment_meta_wide)

# Figure 5

In [ ]:
nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
d = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'#PASS_MSCV_Vuckovic2020_w_unconstrained.alpha_mix.txt'
alpha = pd.concat([pd.read_csv(d + t + '_w_unconstrained.alpha_mix.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits])
alpha.to_csv(
    '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/alpha_mix_trait_specific.csv',
    index = False
)


# Figure 4 via meta-analysis

In [ ]:
from statsmodels.stats.meta_analysis import combine_effects

nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'
combined_alpha_mix_estimates = pd.concat([pd.read_csv(f'{dir}{t}_w_unconstrained.alpha_mix.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits])
#combine_effects(
#    effect = combined_alpha_mix_estimates.w_point_estimate,
#    variance = combined_alpha_mix_estimates.w_se ** 2
#).summary_frame()
combine_effects(
    effect = combined_alpha_mix_estimates.alpha_point_estimate,
    variance = combined_alpha_mix_estimates.alpha_se ** 2
).summary_frame()

## Heterogeneity test

In [ ]:
#combine_effects(
#    effect = combined_alpha_mix_estimates.w_point_estimate,
#    variance = combined_alpha_mix_estimates.w_se ** 2
#).summary_frame()
I_statistic = np.sum((((combined_alpha_mix_estimates.w_point_estimate - 0.998356) / combined_alpha_mix_estimates.w_se) ** 2))
from scipy.stats import chi2
#chi2.cdf(1.96 ** 2, 1)
1 - chi2.cdf(I_statistic, 49)

combine_effects(
    effect = combined_alpha_mix_estimates.alpha_point_estimate,
    variance = combined_alpha_mix_estimates.alpha_se ** 2
).summary_frame()
I_statistic = np.sum((((combined_alpha_mix_estimates.alpha_point_estimate -0.310535) / combined_alpha_mix_estimates.alpha_se) ** 2))
#1 - chi2.cdf(I_statistic, 49)

In [ ]:
from statsmodels.stats.meta_analysis import combine_effects

nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'
combined_alpha_mix_estimates = pd.concat([pd.read_csv(f'{dir}{t}_w_unconstrained.alpha_mix_varying_w.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits]).assign(w = lambda df: np.round(df.w, 2))
loss_summed = combined_alpha_mix_estimates.groupby('w').sum(numeric_only = True).reset_index()[['w', 'loss_point_estimate']]
px.line(loss_summed, x = 'w', y = 'loss_point_estimate')
meta_analyzed_alpha_df_list = []
for w in np.round(np.linspace(0, 1, 101), 2):
    current_w_params = combined_alpha_mix_estimates.query(f'w == {w}')
    wow = combine_effects(
        effect = current_w_params.alpha_point_estimate, 
        variance = current_w_params.alpha_se** 2
    ).summary_frame(
    ).loc[
        ['random effect']][['eff', 'ci_low', 'ci_upp']
    ]
    meta_analyzed_alpha_df_list.append(wow.assign(w = w))
meta_analyzed_alpha_df = pd.concat(meta_analyzed_alpha_df_list).merge(loss_summed, on = 'w')
meta_analyzed_alpha_df.to_csv(dir + 'alpha_mix_varying_w_meta.csv', index = False)

# Figure via meta-analysis on common domain

In [ ]:
from statsmodels.stats.meta_analysis import combine_effects

nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'
combined_alpha_mix_estimates = pd.concat([pd.read_csv(f'{dir}{t}_w_unconstrained_common_domain.alpha_mix.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits])
#combine_effects(
#    effect = combined_alpha_mix_estimates.query('trait != "UKB_460K.biochemistry_Phosphate"').w_mean,
#    variance = combined_alpha_mix_estimates.query('trait != "UKB_460K.biochemistry_Phosphate"').w_se ** 2
#).summary_frame()
#combine_effects(
#    effect = combined_alpha_mix_estimates.w_point_estimate,
#    variance = combined_alpha_mix_estimates.w_se ** 2
#).summary_frame()
#combine_effects(
#    effect = combined_alpha_mix_estimates.query('trait != "UKB_460K.biochemistry_Phosphate"').alpha_mean,
#    variance = combined_alpha_mix_estimates.query('trait != "UKB_460K.biochemistry_Phosphate"').alpha_se ** 2
#).summary_frame()
combine_effects(
    effect = combined_alpha_mix_estimates.alpha_point_estimate,
    variance = combined_alpha_mix_estimates.alpha_se ** 2
).summary_frame()

In [ ]:
from statsmodels.stats.meta_analysis import combine_effects

nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'
combined_alpha_mix_estimates = pd.concat([pd.read_csv(f'{dir}{t}_w_unconstrained_common_domain.alpha_mix_varying_w.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits]).assign(w = lambda df: np.round(df.w, 2))
loss_summed = combined_alpha_mix_estimates.groupby('w').sum(numeric_only = True).reset_index()[['w', 'loss_point_estimate']]
px.line(loss_summed, x = 'w', y = 'loss_point_estimate')
meta_analyzed_alpha_df_list = []
for w in np.round(np.linspace(0, 1, 101), 2):
    current_w_params = combined_alpha_mix_estimates.query(f'w == {w}')
    wow = combine_effects(
        effect = current_w_params.alpha_point_estimate, 
        variance = current_w_params.alpha_se** 2
    ).summary_frame(
    ).loc[
        ['random effect']][['eff', 'ci_low', 'ci_upp']
    ]
    meta_analyzed_alpha_df_list.append(wow.assign(w = w))
meta_analyzed_alpha_df = pd.concat(meta_analyzed_alpha_df_list).merge(loss_summed, on = 'w')
meta_analyzed_alpha_df.to_csv(dir + 'alpha_mix_varying_w_meta_common_domain.csv', index = False)
px.line(meta_analyzed_alpha_df, x = 'w', y = 'loss_point_estimate')

In [ ]:
from statsmodels.stats.meta_analysis import combine_effects

nh2_selected_traits = set(pd.read_csv('/n/groups/price/jordan/h2xancestry/data/sumstats/jordans_trait_list.txt', sep = '\t').iloc[:,0].tolist())
dir = '/n/groups/price/jordan/h2xancestry/data/real_traits_2/baselineLD2.2_plus_maf_grid/'
combined_alpha_mix_estimates = pd.concat([pd.read_csv(f'{dir}{t}_w_unconstrained_common_domain.alpha_mix_varying_w.txt', sep = '\t').assign(trait = t) for t in nh2_selected_traits]).assign(w = lambda df: np.round(df.w, 2)).query('trait != "UKB_460K.biochemistry_Phosphate"')
loss_summed = combined_alpha_mix_estimates.groupby('w').sum(numeric_only = True).reset_index()[['w', 'loss_point_estimate']]
px.line(loss_summed, x = 'w', y = 'loss_point_estimate')
meta_analyzed_alpha_df_list = []
for w in np.round(np.linspace(0, 1, 101), 2):
    current_w_params = combined_alpha_mix_estimates.query(f'w == {w}')
    wow = combine_effects(
        effect = current_w_params.alpha_point_estimate, 
        variance = current_w_params.alpha_se** 2
    ).summary_frame(
    ).loc[
        ['random effect']][['eff', 'ci_low', 'ci_upp']
    ]
    meta_analyzed_alpha_df_list.append(wow.assign(w = w))
meta_analyzed_alpha_df = pd.concat(meta_analyzed_alpha_df_list).merge(loss_summed, on = 'w')
meta_analyzed_alpha_df.to_csv(dir + 'alpha_mix_varying_w_meta_common_domain_no_phosphate.csv', index = False)
px.line(meta_analyzed_alpha_df, x = 'w', y = 'loss_point_estimate')